<a href="https://colab.research.google.com/github/prof-cs-ohamdas/ML_DL_IDS_UAV/blob/main/ML_IDS_UAV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
import warnings
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

In [95]:
# Try import pennylane (QML). If not available, we'll fall back to classical surrogate.
try:
    import pennylane as qml
    from pennylane import numpy as pnp
    from pennylane.optimize import AdamOptimizer
    QML_AVAILABLE = True
    print("[INFO] pennylane imported: QML enabled.")
except Exception:
    qml = None
    pnp = None
    AdamOptimizer = None
    QML_AVAILABLE = False
    warnings.warn(
        "pennylane not available — QML will fall back to a classical MLP surrogate. "
        "To enable QML, install pennylane: pip install pennylane"
    )

[INFO] pennylane imported: QML enabled.


Data Set load from Github

In [96]:
import pandas as pd

url = "https://raw.githubusercontent.com/prof-cs-ohamdas/ML_DL_IDS_UAV/main/drone_communication_dataset.csv"
df = pd.read_csv(url)

print(f"[INFO] Dataset loaded: {df.shape[0]} rows, {df.shape[1]} columns")


[INFO] Dataset loaded: 45289 rows, 35 columns


In [97]:
!git clone https://github.com/prof-cs-ohamdas/ML_DL_IDS_UAV.git
%cd ML_DL_IDS_UAV

import pandas as pd
df = pd.read_csv("drone_communication_dataset.csv")
print(f"[INFO] Dataset loaded: {df.shape[0]} rows, {df.shape[1]} columns")


fatal: destination path 'ML_DL_IDS_UAV' already exists and is not an empty directory.
/content/ML_DL_IDS_UAV
[INFO] Dataset loaded: 45289 rows, 35 columns


In [98]:
from google.colab import data_table
data_table.enable_dataframe_formatter()

df


,timestamp,signal_strength,packet_loss_rate,round_trip_time,communication_protocol,frequency_band,encryption_type,drone_gps_coordinates,altitude,speed_trajectory,...,intrusion_detection_flags,temporal_patterns,label_normal,label_spoofing,label_mitm,label_ddos,label_gps_spoofing,label_malware,label_jamming,label_protocol_exploit
0,2019-11-01 00:00:00,-60,0,50,ZigBee,5.0,AES,"(-11.69864152850468, 86.89778588010734)",291,60,...,0,0,0,0,0,0,0,0,0,0
1,2019-11-01 01:00:00,-64,0,50,LoRa,2.4,AES,"(66.98918792506822, -140.91891468822496)",100,48,...,0,0,0,0,0,0,0,0,0,0
2,2019-11-01 02:00:00,-60,0,194,ZigBee,2.4,AES,"(-31.711586784829194, 36.5632095665857)",226,30,...,0,0,0,0,0,0,0,0,0,0
3,2019-11-01 03:00:00,-60,0,50,Wi-Fi,2.4,AES,"(3.5943733073538198, 74.7992608739666)",100,30,...,0,0,1,0,0,0,0,0,1,0
4,2019-11-01 04:00:00,-69,0,50,Wi-Fi,2.4,Plain-text,"(41.4738797417983, -100.00817301602393)",100,30,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45284,2024-12-30 20:00:00,-60,0,50,LoRa,915.0,Plain-text,"(62.250807704044774, -151.05594484833514)",100,30,...,0,0,0,0,0,0,0,0,0,0
45285,2024-12-30 21:00:00,-66,0,50,ZigBee,2.4,RSA,"(-44.694536026285874, 164.52171076223817)",100,30,...,0,0,0,0,0,0,0,0,0,0
45286,2024-12-30 22:00:00,-60,12,50,ZigBee,2.4,RSA,"(-89.91726166541113, -120.42483221247912)",100,30,...,0,1,0,0,0,1,1,0,1,0
45287,2024-12-30 23:00:00,-60,0,50,Wi-Fi,2.4,Plain-text,"(-55.27640423530802, 61.96296050101765)",100,30,...,0,1,0,0,0,0,0,0,0,0


In [99]:
# -------------------------
# 2. Inspect label columns
# -------------------------
label_cols = [c for c in df.columns if c.startswith("label_")]
if not label_cols:
    raise ValueError("No label_* columns found in dataset. Need label columns for supervised training.")
print(f"[INFO] label columns detected: {label_cols}")


[INFO] label columns detected: ['label_normal', 'label_spoofing', 'label_mitm', 'label_ddos', 'label_gps_spoofing', 'label_malware', 'label_jamming', 'label_protocol_exploit']


In [100]:
# -------------------------
# 3. Create multi-class target (attack_type)
# -------------------------
# Each row is assumed to have a one-hot label across label_* columns; idxmax gives the label name.
df["attack_type_raw"] = df[label_cols].idxmax(axis=1)  # e.g. 'label_ddos'
df["attack_type"] = df["attack_type_raw"].str.replace("label_", "", regex=False)


In [101]:
# -------------------------
# 4. Create binary target for QML: normal vs attack
#    Strategy: try to find explicit 'normal' label; otherwise assume the most frequent label is 'normal'
# -------------------------
normal_label_col = next((c for c in label_cols if "normal" in c.lower()), None)
if normal_label_col:
    print(f"[INFO] Found explicit normal label column: {normal_label_col}")
    df["is_attack"] = (df[normal_label_col] == 0).astype(int)  # 1 -> attack, 0 -> normal
else:
    # No explicit 'label_normal' column. We'll assume the most frequent label is 'normal'.
    most_freq_label = df["attack_type"].mode().iloc[0]
    print(f"[WARN] No explicit 'label_normal' column. Treating most frequent label '{most_freq_label}' as NORMAL.")
    df["is_attack"] = (df["attack_type"] != most_freq_label).astype(int)


[INFO] Found explicit normal label column: label_normal


In [102]:
# -------------------------
# 5. Drop non-useful columns
# -------------------------
cols_to_drop = []
for c in ["timestamp", "drone_gps_coordinates", "attack_type_raw"]:
    if c in df.columns:
        cols_to_drop.append(c)

# We'll keep numeric/categorical features; drop label_* originals after deriving targets
cols_to_drop += label_cols
df = df.drop(columns=cols_to_drop)
print(f"[INFO] Dropped columns: {cols_to_drop}")


[INFO] Dropped columns: ['timestamp', 'drone_gps_coordinates', 'attack_type_raw', 'label_normal', 'label_spoofing', 'label_mitm', 'label_ddos', 'label_gps_spoofing', 'label_malware', 'label_jamming', 'label_protocol_exploit']


In [103]:
# -------------------------
# 6. Prepare features and encode categorical columns
# -------------------------
# Identify object (text) columns except 'attack_type' (we'll encode it separately)
categorical_cols = df.select_dtypes(include=["object"]).columns.tolist()
if "attack_type" in categorical_cols:
    categorical_cols.remove("attack_type")

print(f"[INFO] Categorical columns to encode: {categorical_cols}")

# One-hot encode categorical cols (safe for models)
df_enc = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

# Final feature matrix X and labels y_multi (multi-class) and y_bin (binary)
X = df_enc.drop(columns=["attack_type", "is_attack"])
y_multi = df_enc["attack_type"]
y_bin = df_enc["is_attack"]

# Encode multi-class labels to integers
le = LabelEncoder()
y_multi_enc = le.fit_transform(y_multi)
print(f"[INFO] Multi-class labels found: {list(le.classes_)}")

[INFO] Categorical columns to encode: ['communication_protocol', 'encryption_type']
[INFO] Multi-class labels found: ['ddos', 'gps_spoofing', 'jamming', 'malware', 'mitm', 'normal', 'protocol_exploit', 'spoofing']


In [104]:
# -------------------------
# 7. Train/test split
# -------------------------
# Classical (multi-class) models
X_train_m, X_test_m, y_train_m, y_test_m = train_test_split(
    X, y_multi_enc, test_size=0.20, random_state=42, stratify=y_multi_enc
)

# QML (binary) - use the same feature set but we'll restrict to smaller feature subset for QNN
X_train_b, X_test_b, y_train_b, y_test_b = train_test_split(
    X, y_bin, test_size=0.20, random_state=42, stratify=y_bin
)

# Standardize (important for SVM and many models)
scaler = StandardScaler()
X_train_m_scaled = scaler.fit_transform(X_train_m)
X_test_m_scaled = scaler.transform(X_test_m)

X_train_b_scaled = scaler.transform(X_train_b)  # use same scaler (fit on multi-class training)
X_test_b_scaled = scaler.transform(X_test_b)

print(f"[INFO] Feature dimension: {X_train_m_scaled.shape[1]}")


[INFO] Feature dimension: 27


In [105]:
# -------------------------
# 8. Train classical multi-class models
# -------------------------
models = {
    "Logistic Regression": LogisticRegression(max_iter=2000, random_state=42),
    "KNN": KNeighborsClassifier(n_neighbors=5),
    "SVM": SVC(kernel="rbf", probability=True, random_state=42)
}

results = {}
for name, model in models.items():
    print(f"\n[TRAIN] {name} (multi-class)...")
    model.fit(X_train_m_scaled, y_train_m)
    preds = model.predict(X_test_m_scaled)
    acc = accuracy_score(y_test_m, preds)
    results[name] = {"model": model, "accuracy": acc, "y_pred": preds}
    print(f"[RESULT] {name} accuracy: {acc:.4f}")
    print(classification_report(y_test_m, preds, target_names=le.classes_))



[TRAIN] Logistic Regression (multi-class)...
[RESULT] Logistic Regression accuracy: 0.7250
                  precision    recall  f1-score   support

            ddos       0.00      0.00      0.00       605
    gps_spoofing       0.00      0.00      0.00       286
         jamming       0.00      0.00      0.00       429
         malware       0.00      0.00      0.00       409
            mitm       0.00      0.00      0.00       237
          normal       0.72      1.00      0.84      6567
protocol_exploit       0.00      0.00      0.00       118
        spoofing       0.00      0.00      0.00       407

        accuracy                           0.72      9058
       macro avg       0.09      0.12      0.11      9058
    weighted avg       0.53      0.72      0.61      9058


[TRAIN] KNN (multi-class)...


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


[RESULT] KNN accuracy: 0.6862
                  precision    recall  f1-score   support

            ddos       0.09      0.03      0.05       605
    gps_spoofing       0.03      0.01      0.01       286
         jamming       0.05      0.01      0.02       429
         malware       0.07      0.01      0.02       409
            mitm       0.04      0.00      0.01       237
          normal       0.73      0.94      0.82      6567
protocol_exploit       0.00      0.00      0.00       118
        spoofing       0.04      0.00      0.00       407

        accuracy                           0.69      9058
       macro avg       0.13      0.13      0.12      9058
    weighted avg       0.54      0.69      0.60      9058


[TRAIN] SVM (multi-class)...
[RESULT] SVM accuracy: 0.7250
                  precision    recall  f1-score   support

            ddos       0.00      0.00      0.00       605
    gps_spoofing       0.00      0.00      0.00       286
         jamming       0.00      0.0

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
# -------------------------
# 9. QML binary classifier (QNN) OR MLP surrogate if pennylane unavailable
#    We'll use a small subset of features for the QNN (e.g., first 3 principal features)
# -------------------------
# Choose k features for the QNN (low-dim). We'll pick the first 3 numeric columns (after encoding).
k_qnn = 3
if X_train_b_scaled.shape[1] < k_qnn:
    k_qnn = X_train_b_scaled.shape[1]

X_train_q = X_train_b_scaled[:, :k_qnn]
X_test_q = X_test_b_scaled[:, :k_qnn]

if QML_AVAILABLE:
    print("\n[QML] Training Quantum Neural Network (binary)...")
    n_qubits = k_qnn
    dev = qml.device("default.qubit", wires=n_qubits)

    # Variational circuit: AngleEmbedding + StronglyEntanglingLayers
    def quantum_circuit(params, x=None):
        # angle embedding expects length==n_qubits
        x_padded = np.resize(x, n_qubits)  # truncate or pad
        qml.AngleEmbedding(x_padded, wires=range(n_qubits))
        qml.StronglyEntanglingLayers(params, wires=range(n_qubits))
        return qml.expval(qml.PauliZ(0))

    n_layers = 2
    params_shape = qml.StronglyEntanglingLayers.shape(n_layers=n_layers, n_wires=n_qubits)
    params = pnp.random.random(params_shape, requires_grad=True)

    @qml.qnode(dev, interface="autograd")
    def qnode(params, x):
        return quantum_circuit(params, x)

    def qnn_predict_prob(params, x):
        # map expectation [-1,1] -> [0,1]
        return (qnode(params, x) + 1) / 2

    # Prepare pnp arrays
    X_train_q_p = pnp.array(X_train_q, requires_grad=False)
    X_test_q_p = pnp.array(X_test_q, requires_grad=False)
    y_train_q_p = pnp.array(y_train_b.values, requires_grad=False)

    opt = AdamOptimizer(stepsize=0.2)

    def cost(params, X, y):
        preds = pnp.array([qnn_predict_prob(params, X[i]) for i in range(len(X))])
        return pnp.mean((preds - y) ** 2)

    epochs = 15
    for epoch in range(epochs):
        params = opt.step(lambda v: cost(v, X_train_q_p, y_train_q_p), params)
        if (epoch + 1) % 5 == 0 or epoch == 0:
            c = cost(params, X_train_q_p, y_train_q_p)
            print(f" Epoch {epoch+1}/{epochs}  cost: {c:.4f}")

    # Evaluate QNN
    q_probs = [qnn_predict_prob(params, x) for x in X_test_q_p]
    q_preds = [1 if float(p) > 0.5 else 0 for p in q_probs]
    q_acc = accuracy_score(y_test_b, q_preds)
    print(f"\n[QNN RESULT] Binary QNN accuracy: {q_acc:.4f}")
    print(classification_report(y_test_b, q_preds))
    results["Quantum QNN (binary)"] = {"model": "pennylane_qnn", "accuracy": q_acc, "y_pred": q_preds}
else:
    # Fallback: small MLP surrogate
    print("\n[QML fallback] pennylane unavailable: training MLP surrogate (binary)...")
    mlp = MLPClassifier(hidden_layer_sizes=(64, 32), max_iter=300, random_state=42)
    mlp.fit(X_train_q, y_train_b)
    mlp_preds = mlp.predict(X_test_q)
    mlp_acc = accuracy_score(y_test_b, mlp_preds)
    print(f"[MLP surrogate] Binary accuracy: {mlp_acc:.4f}")
    print(classification_report(y_test_b, mlp_preds))
    results["QNN surrogate (MLP)"] = {"model": mlp, "accuracy": mlp_acc, "y_pred": mlp_preds}



[QML] Training Quantum Neural Network (binary)...


In [ ]:
# -------------------------
# 10. Sanity checks
# -------------------------
print("\n[Sanity Checks]")
for name, info in results.items():
    pred = info["y_pred"]
    if len(pred) != X_test_m_scaled.shape[0] and ("binary" not in name.lower() and "qnn" not in name.lower() and "surrogate" not in name.lower()):
        print(f" [WARN] {name} prediction length ({len(pred)}) != multi-class test size ({X_test_m_scaled.shape[0]})")
    else:
        print(f" [OK] {name} predictions ready (accuracy {info['accuracy']:.4f})")

print("\n[Done] IDS training & evaluation complete.")


**Graph Neural Networks (GNNs)**

In [ ]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
import torch.nn.functional as F

In [ ]:
import pandas as pd

url = "https://raw.githubusercontent.com/prof-cs-ohamdas/ML_DL_IDS_UAV/main/drone_communication_dataset.csv"
df = pd.read_csv(url)

print(f"[INFO] Dataset loaded: {df.shape[0]} rows, {df.shape[1]} columns")

In [ ]:
# Drop non-useful columns
drop_cols = ["timestamp", "drone_gps_coordinates"]
df = df.drop(columns=[c for c in drop_cols if c in df.columns])


In [ ]:
# Merge labels
label_cols = [c for c in df.columns if c.startswith("label_")]
df["attack_type"] = df[label_cols].idxmax(axis=1)
df["attack_type"] = df["attack_type"].str.replace("label_", "")
df = df.drop(columns=label_cols)

In [ ]:
# Encode categorical features
categorical_cols = df.select_dtypes(include=["object"]).columns.drop("attack_type")
df = pd.get_dummies(df, columns=categorical_cols)

In [ ]:
# Encode target
le = LabelEncoder()
df["attack_type"] = le.fit_transform(df["attack_type"])

X = df.drop(columns=["attack_type"]).values
y = df["attack_type"].values

In [ ]:
# Standardize
scaler = StandardScaler()
X = scaler.fit_transform(X)


In [ ]:
# -------------------------
# 2. Build Graph
# -------------------------
# For simplicity: connect each node to its k nearest neighbors
from sklearn.neighbors import kneighbors_graph
import numpy as np

k = 5
adj = kneighbors_graph(X, k, mode="connectivity", include_self=False)
edge_index = torch.tensor(np.array(adj.nonzero()), dtype=torch.long)

# Convert to PyTorch Geometric Data
data = Data(
    x=torch.tensor(X, dtype=torch.float),
    y=torch.tensor(y, dtype=torch.long),
    edge_index=edge_index
)


In [ ]:
# Train/test split
num_nodes = data.num_nodes
train_mask = torch.zeros(num_nodes, dtype=torch.bool)
test_mask = torch.zeros(num_nodes, dtype=torch.bool)

train_idx, test_idx = train_test_split(range(num_nodes), test_size=0.2, stratify=y, random_state=42)
train_mask[train_idx] = True
test_mask[test_idx] = True
data.train_mask = train_mask
data.test_mask = test_mask

In [ ]:
# -------------------------
# 3. Define GNN Model
# -------------------------
class GNN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GNN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return x

model = GNN(in_channels=data.num_node_features, hidden_channels=32, out_channels=len(le.classes_))

optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
loss_fn = torch.nn.CrossEntropyLoss()

In [ ]:
# -------------------------
# 4. Train
# -------------------------
for epoch in range(50):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = loss_fn(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")

In [ ]:
# -------------------------
# 5. Test
# -------------------------
model.eval()
preds = out.argmax(dim=1)
correct = preds[data.test_mask].eq(data.y[data.test_mask]).sum().item()
acc = correct / data.test_mask.sum().item()
print(f"Test Accuracy: {acc:.4f}")

**H-QNN**

In [ ]:
import sys
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

In [ ]:
# Try imports for PennyLane; give helpful message if missing.
try:
    import pennylane as qml
    from pennylane import numpy as pnp
    from pennylane import qnn as qml_qnn  # optional alias
except Exception as e:
    print("ERROR: pennylane is required for H-QNN. Install with: pip install pennylane")
    raise

In [ ]:
# -------------------------
# 1. Load dataset (from GitHub URL)
# -------------------------
url = "https://raw.githubusercontent.com/prof-cs-ohamdas/ML_DL_IDS_UAV/main/drone_communication_dataset.csv"
df = pd.read_csv(url)
print(f"[INFO] Dataset loaded: {df.shape[0]} rows, {df.shape[1]} columns")


In [ ]:
# -------------------------
# 2. Basic cleanup & label handling
# -------------------------
# Drop obviously useless columns if present
drop_cols = ["timestamp", "drone_gps_coordinates"]
df = df.drop(columns=[c for c in drop_cols if c in df.columns])

# Find label columns: either many label_* columns or single attack_type/label
label_cols = [c for c in df.columns if c.startswith("label_")]

if len(label_cols) > 0:
    # one-hot style labels -> collapse to single text label
    df["attack_type"] = df[label_cols].idxmax(axis=1)
    df["attack_type"] = df["attack_type"].str.replace("label_", "", regex=False)
    df = df.drop(columns=label_cols)
elif "attack_type" in df.columns:
    # already present
    pass
elif "label" in df.columns:
    df = df.rename(columns={"label": "attack_type"})
else:
    raise ValueError("No label columns found. Expecting 'label_*' or 'attack_type' or 'label'.")


In [ ]:
# -------------------------
# 3. Encode categorical features and target
# -------------------------
# Separate object dtype columns except 'attack_type'
categorical_cols = df.select_dtypes(include=["object"]).columns.tolist()
if "attack_type" in categorical_cols:
    categorical_cols.remove("attack_type")

if len(categorical_cols) > 0:
    df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

# Encode target labels to integers
le = LabelEncoder()
df["attack_type"] = le.fit_transform(df["attack_type"])

# Final feature matrix and labels
X = df.drop(columns=["attack_type"]).values
y = df["attack_type"].values
print(f"[INFO] After encoding: X shape = {X.shape}, classes = {list(le.classes_)}")


In [ ]:
# -------------------------
# 4. Scale numeric features
# -------------------------
scaler = StandardScaler()
X = scaler.fit_transform(X)

# -------------------------
# 5. Subset and train/test split (we use a smaller training subset for Q-sim speed)
# -------------------------
# Full test split
X_train_full, X_test, y_train_full, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42, stratify=y
)

# Choose a smaller training size for manageable quantum simulation (adjustable)
max_train_samples = 2000  # change if you want to train on more samples
if X_train_full.shape[0] > max_train_samples:
    # stratified downsample
    frac = max_train_samples / X_train_full.shape[0]
    X_train, _, y_train, _ = train_test_split(
        X_train_full, y_train_full, train_size=frac, random_state=42, stratify=y_train_full
    )
    print(f"[INFO] Training on a downsampled subset: {X_train.shape[0]} samples (of {X_train_full.shape[0]} full).")
else:
    X_train = X_train_full
    y_train = y_train_full
    print(f"[INFO] Training on full training set: {X_train.shape[0]} samples.")

# Convert to torch tensors
X_train_t = torch.tensor(X_train, dtype=torch.float32)
y_train_t = torch.tensor(y_train, dtype=torch.long)
X_test_t = torch.tensor(X_test, dtype=torch.float32)
y_test_t = torch.tensor(y_test, dtype=torch.long)

# Dataloader
batch_size = 32
train_loader = DataLoader(TensorDataset(X_train_t, y_train_t), batch_size=batch_size, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test_t, y_test_t), batch_size=batch_size, shuffle=False)


In [ ]:
# -------------------------
# 6. Q-circuit & TorchLayer definition (safe & consistent)
# -------------------------
# Decide how many qubits to use. We'll compress classical features into n_qubits with a linear layer.
n_qubits = 4
n_layers = 1  # number of variational layers (small to keep simulation reasonable)

dev = qml.device("default.qubit", wires=n_qubits)

# QNode: inputs will be a vector of length n_qubits (per sample). Return n_qubits expectation values.
@qml.qnode(dev, interface="torch")
def qnode(inputs, weights):
    # angle embedding: inputs length must be <= n_qubits; we'll ensure shape before calling
    qml.templates.AngleEmbedding(inputs, wires=range(n_qubits))
    qml.templates.StronglyEntanglingLayers(weights, wires=range(n_qubits))
    # return a tuple/list of expectation values (one per qubit)
    return tuple(qml.expval(qml.PauliZ(i)) for i in range(n_qubits))

# Weight shapes must match (n_layers, n_qubits, 3)
weight_shapes = {"weights": (n_layers, n_qubits, 3)}
qlayer = qml.qnn.TorchLayer(qnode, weight_shapes)

In [ ]:
# -------------------------
# 7. Hybrid model: classical compressor -> qlayer -> classifier
# -------------------------
class HQNNModel(nn.Module):
    def __init__(self, in_features, n_qubits, n_classes):
        super().__init__()
        # compress arbitrary in_features -> n_qubits so quantum layer gets exactly n_qubits inputs
        self.fc_pre = nn.Linear(in_features, n_qubits)
        self.qlayer = qlayer      # qlayer expects vector length == n_qubits
        self.fc_post = nn.Linear(n_qubits, n_classes)

    def forward(self, x):
        # Ensure 2D: (batch_size, features)
        x = x.view(x.shape[0], -1)
        # classical compression
        x = torch.tanh(self.fc_pre(x))   # -> (batch, n_qubits)
        # quantum layer: TorchLayer accepts (batch, n_qubits) and returns (batch, n_qubits)
        x = self.qlayer(x)
        # final classifier
        x = self.fc_post(x)
        return x

in_features = X_train.shape[1]
n_classes = len(np.unique(y))
model = HQNNModel(in_features=in_features, n_qubits=n_qubits, n_classes=n_classes)
print(f"[INFO] Model created: in_features={in_features}, n_qubits={n_qubits}, n_classes={n_classes}")

In [ ]:
# -------------------------
# 8. Training loop (minibatches)
# -------------------------
device = torch.device("cpu")
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

n_epochs = 6
for epoch in range(1, n_epochs + 1):
    model.train()
    epoch_loss = 0.0
    correct = 0
    total = 0
    for batch_x, batch_y in train_loader:
        batch_x = batch_x.to(device)
        batch_y = batch_y.to(device)

        optimizer.zero_grad()
        outputs = model(batch_x)            # shape (batch_size, n_classes)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item() * batch_x.size(0)
        preds = outputs.argmax(dim=1)
        correct += (preds == batch_y).sum().item()
        total += batch_x.size(0)

    epoch_loss /= total
    train_acc = correct / total
    print(f"[Epoch {epoch}/{n_epochs}] Loss: {epoch_loss:.4f}, Train Acc: {train_acc:.4f}")


**XGBoost, LightGBM, CatBoost**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report


In [ ]:
# Try importing XGBoost
try:
    from xgboost import XGBClassifier
    xgb_available = True
except ImportError:
    print("Warning: XGBoost not installed. Skipping XGBoost.")
    xgb_available = False

In [ ]:
# Try importing LightGBM
try:
    from lightgbm import LGBMClassifier
    lgbm_available = True
except ImportError:
    print("Warning: LightGBM not installed. Skipping LightGBM.")
    lgbm_available = False

In [ ]:
# Try importing CatBoost
try:
    from catboost import CatBoostClassifier
    catboost_available = True
except ImportError:
    print("Warning: CatBoost not installed. Skipping CatBoost.")
    catboost_available = False

In [ ]:
# -------------------------
# 1. Load and preprocess dataset
# -------------------------
url = "https://raw.githubusercontent.com/prof-cs-ohamdas/ML_DL_IDS_UAV/main/drone_communication_dataset.csv"
df = pd.read_csv(url)
print(f"[INFO] Dataset loaded: {df.shape[0]} rows, {df.shape[1]} columns")

# Drop non-useful columns
drop_cols = ["timestamp", "drone_gps_coordinates"]
df = df.drop(columns=[c for c in drop_cols if c in df.columns])

In [ ]:
# Merge attack labels
label_cols = [c for c in df.columns if c.startswith("label_")]
df["attack_type"] = df[label_cols].idxmax(axis=1)
df["attack_type"] = df["attack_type"].str.replace("label_", "")
df = df.drop(columns=label_cols)


In [ ]:
# Encode categorical features
categorical_cols = df.select_dtypes(include=["object"]).columns.drop("attack_type")
df = pd.get_dummies(df, columns=categorical_cols)

In [ ]:
# Encode target
le = LabelEncoder()
df["attack_type"] = le.fit_transform(df["attack_type"])

X = df.drop(columns=["attack_type"]).values
y = df["attack_type"].values


In [ ]:
# Standardize
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [ ]:
# -------------------------
# 2. Boosting Models
# -------------------------
models = {}

if xgb_available:
    models["XGBoost"] = XGBClassifier(use_label_encoder=False, eval_metric="mlogloss")

if lgbm_available:
    models["LightGBM"] = LGBMClassifier()

if catboost_available:
    models["CatBoost"] = CatBoostClassifier(verbose=0)


In [ ]:
# -------------------------
# 3. Train and Evaluate
# -------------------------
for name, model in models.items():
    print(f"\n================ {name} ================")
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    acc = accuracy_score(y_test, preds)
    print(f"{name} Accuracy: {acc:.4f}")
    print(classification_report(y_test, preds, target_names=le.classes_))


**Deep Neural Networks (DNNs)**

In [4]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, hamming_loss
from sklearn.neural_network import MLPClassifier
from sklearn.multioutput import MultiOutputClassifier
import joblib
import matplotlib.pyplot as plt

In [5]:
# ===============================
# 1. Load dataset from GitHub
# ===============================
import pandas as pd

url = "https://raw.githubusercontent.com/prof-cs-ohamdas/ML_DL_IDS_UAV/main/drone_communication_dataset.csv"
df = pd.read_csv(url)

print(f"[INFO] Dataset loaded from GitHub: {df.shape[0]} rows, {df.shape[1]} columns")

[INFO] Dataset loaded from GitHub: 45289 rows, 35 columns


In [6]:
# ===============================
# 2. Clone the repo (optional, if you want local copy in Colab)
# ===============================
!git clone https://github.com/prof-cs-ohamdas/ML_DL_IDS_UAV.git
%cd ML_DL_IDS_UAV

# Reload from local file (safer if large dataset)
df = pd.read_csv("drone_communication_dataset.csv")
print(f"[INFO] Dataset loaded locally: {df.shape[0]} rows, {df.shape[1]} columns")


fatal: destination path 'ML_DL_IDS_UAV' already exists and is not an empty directory.
/content/ML_DL_IDS_UAV
[INFO] Dataset loaded locally: 45289 rows, 35 columns


In [7]:
# ===============================
# 3. Enable Colab interactive table
# ===============================
from google.colab import data_table
data_table.enable_dataframe_formatter()

df.head()   # preview in interactive table


,timestamp,signal_strength,packet_loss_rate,round_trip_time,communication_protocol,frequency_band,encryption_type,drone_gps_coordinates,altitude,speed_trajectory,...,intrusion_detection_flags,temporal_patterns,label_normal,label_spoofing,label_mitm,label_ddos,label_gps_spoofing,label_malware,label_jamming,label_protocol_exploit
0,2019-11-01 00:00:00,-60,0,50,ZigBee,5.0,AES,"(-11.69864152850468, 86.89778588010734)",291,60,...,0,0,0,0,0,0,0,0,0,0
1,2019-11-01 01:00:00,-64,0,50,LoRa,2.4,AES,"(66.98918792506822, -140.91891468822496)",100,48,...,0,0,0,0,0,0,0,0,0,0
2,2019-11-01 02:00:00,-60,0,194,ZigBee,2.4,AES,"(-31.711586784829194, 36.5632095665857)",226,30,...,0,0,0,0,0,0,0,0,0,0
3,2019-11-01 03:00:00,-60,0,50,Wi-Fi,2.4,AES,"(3.5943733073538198, 74.7992608739666)",100,30,...,0,0,1,0,0,0,0,0,1,0
4,2019-11-01 04:00:00,-69,0,50,Wi-Fi,2.4,Plain-text,"(41.4738797417983, -100.00817301602393)",100,30,...,0,0,0,0,0,0,0,0,0,0


In [8]:
# ===============================
# 4. Preprocess + IDS (MLP/DNN)
# ===============================
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.neural_network import MLPClassifier
from sklearn.multioutput import MultiOutputClassifier
import joblib


In [9]:
# Detect label columns (multi-label setup)
label_cols = [c for c in df.columns if c.lower().startswith("label_")]
if not label_cols:
    label_cols = [df.columns[-1]]
print(f"[INFO] Using label columns: {label_cols}")

X = df.drop(columns=label_cols)
Y = df[label_cols].copy()

[INFO] Using label columns: ['label_normal', 'label_spoofing', 'label_mitm', 'label_ddos', 'label_gps_spoofing', 'label_malware', 'label_jamming', 'label_protocol_exploit']


In [10]:
# Encode categorical Y if needed
for col in label_cols:
    if Y[col].dtype == object:
        le = LabelEncoder()
        Y[col] = le.fit_transform(Y[col].astype(str))


In [11]:
# Handle missing values in X
for col in X.columns:
    if X[col].dtype == object:
        X[col] = X[col].fillna(X[col].mode().iloc[0])
    else:
        X[col] = X[col].fillna(X[col].median())

In [12]:
# One-hot encode categorical features
categorical_cols = X.select_dtypes(include=['object','category']).columns.tolist()
if categorical_cols:
    print(f"[INFO] One-hot encoding: {categorical_cols}")
    X = pd.get_dummies(X, columns=categorical_cols, drop_first=True)

[INFO] One-hot encoding: ['timestamp', 'communication_protocol', 'encryption_type', 'drone_gps_coordinates']


In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Step 1: Load dataset
url = "https://raw.githubusercontent.com/prof-cs-ohamdas/ML_DL_IDS_UAV/main/drone_communication_dataset.csv"
df = pd.read_csv(url)

print(f"[INFO] Dataset loaded: {df.shape[0]} rows, {df.shape[1]} columns")

# Step 2: Separate features (X) and labels (y)
# Drop non-numeric columns automatically
X = df.drop(columns=[df.columns[-1]])  # all columns except last
y = df.iloc[:, -1]                     # last column is target

# Keep only numeric columns for features
X = X.select_dtypes(include=["int64", "float64"])

print(f"[INFO] Numeric feature matrix: {X.shape[1]} columns after dropping non-numeric")

# Step 3: Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Step 4: Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

print("[INFO] Shapes:")
print("X_train:", X_train.shape)
print("X_test:", X_test.shape)
print("y_train:", y_train.shape)
print("y_test:", y_test.shape)


[INFO] Dataset loaded: 45289 rows, 35 columns
[INFO] Numeric feature matrix: 30 columns after dropping non-numeric
[INFO] Shapes:
X_train: (36231, 30)
X_test: (9058, 30)
y_train: (36231,)
y_test: (9058,)


In [19]:
# =============================
# IDS on UAV using DNN
# =============================

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical


In [20]:
# -----------------------------
# Step 1: Load dataset
# -----------------------------
url = "https://raw.githubusercontent.com/prof-cs-ohamdas/ML_DL_IDS_UAV/main/drone_communication_dataset.csv"
df = pd.read_csv(url)

print(f"[INFO] Dataset loaded: {df.shape[0]} rows, {df.shape[1]} columns")


[INFO] Dataset loaded: 45289 rows, 35 columns


In [21]:
# -----------------------------
# Step 2: Features (X) and labels (y)
# -----------------------------
X = df.drop(columns=[df.columns[-1]])   # drop last col (label)
y = df.iloc[:, -1]                      # last col is label

# Keep only numeric features
X = X.select_dtypes(include=["int64", "float64"])
print(f"[INFO] Numeric features: {X.shape[1]} columns")


[INFO] Numeric features: 30 columns


In [22]:
# -----------------------------
# Step 3: Encode labels
# -----------------------------
le = LabelEncoder()
y_encoded = le.fit_transform(y)
y_categorical = to_categorical(y_encoded)

print(f"[INFO] Classes found: {len(le.classes_)} -> {list(le.classes_)}")


[INFO] Classes found: 2 -> [np.int64(0), np.int64(1)]


In [23]:
# -----------------------------
# Step 4: Scale features
# -----------------------------
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [24]:
# -----------------------------
# Step 5: Train-test split
# -----------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_categorical, test_size=0.2, random_state=42, stratify=y_categorical
)

print("[INFO] Train/Test split:")
print("X_train:", X_train.shape)
print("X_test:", X_test.shape)

[INFO] Train/Test split:
X_train: (36231, 30)
X_test: (9058, 30)


In [25]:
# -----------------------------
# Step 6: Build DNN model
# -----------------------------
model = Sequential([
    Dense(128, input_dim=X_train.shape[1], activation="relu"),
    Dropout(0.3),
    Dense(64, activation="relu"),
    Dropout(0.3),
    Dense(32, activation="relu"),
    Dense(y_categorical.shape[1], activation="softmax")  # multi-class output
])

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │         3,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 2)              │            66 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,370 (56.13 KB)

 Trainable params: 14,370 (56.13 KB)

 Non-trainable params: 0 (0.00 B)

In [26]:
# -----------------------------
# Step 7: Train model
# -----------------------------
history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=64,
    validation_split=0.2,
    verbose=1
)

Epoch 1/20
453/453 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.9387 - loss: 0.1902 - val_accuracy: 0.9774 - val_loss: 0.1120
Epoch 2/20
453/453 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9802 - loss: 0.1044 - val_accuracy: 0.9774 - val_loss: 0.1091
Epoch 3/20
453/453 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.9787 - loss: 0.1091 - val_accuracy: 0.9774 - val_loss: 0.1102
Epoch 4/20
453/453 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9799 - loss: 0.1008 - val_accuracy: 0.9774 - val_loss: 0.1092
Epoch 5/20
453/453 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9802 - loss: 0.0991 - val_accuracy: 0.9774 - val_loss: 0.1092
Epoch 6/20
453/453 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9783 - loss: 0.1063 - val_accuracy: 0.9774 - val_loss: 0.1099
Epoch 7/20
453/453 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9798 - loss: 0.0990 - val_accuracy: 0.9774 - val_loss: 0.1089
Epoch 8/20
453/453 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9795 - loss: 0.1000 - val_accuracy: 0.

In [27]:
# -----------------------------
# Step 8: Evaluate model
# -----------------------------
loss, acc = model.evaluate(X_test, y_test, verbose=0)
print(f"[RESULT] Test Accuracy: {acc*100:.2f}%")


[RESULT] Test Accuracy: 97.89%


**Federated Learning (FL)**

In [ ]:
# Federated Learning IDS Example for UAV 5G Network
# Using Flower (flwr) and Logistic Regression

import flwr as fl
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [ ]:
# -------------------------
# 4. Start Federated Learning
# -------------------------
from flwr.server.server import ServerConfig
import flwr as fl # Import flwr here as well
import numpy as np # Import numpy

# Define the number of clients
num_clients = 3 # Use the existing variable

# Prepare data for clients (example: split X_train, y_train)
# In a real FL scenario, each client would have its own data
# For simulation, we'll split the training data further.
# The train/test split was already done in previous cells, resulting in X_train_m, y_train_m, X_test_m, y_test_m
# and X_train_b, y_train_b, X_test_b, y_test_b.

# We'll use the binary split data for FL for simplicity (is_attack target)
# Let's split X_train_b and y_train_b into num_clients partitions

X_train_b_split = np.array_split(X_train_b, num_clients)
y_train_b_split = np.array_split(y_train_b, num_clients)

client_data = {str(i): (X_train_b_split[i], y_train_b_split[i]) for i in range(num_clients)}

print(f"[INFO] Data split into {num_clients} partitions for FL clients before starting simulation.")


def client_fn(cid: str):
    # client_data is now defined before this function is used in start_simulation
    return UAVClient(cid, client_data[int(cid)])


strategy = fl.server.strategy.FedAvg()
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=num_clients,
    config=ServerConfig(num_rounds=5),
    strategy=strategy,
)

In [ ]:
# -------------------------
# 6. Prepare features and encode categorical columns
# -------------------------
# Identify object (text) columns except 'attack_type' (we'll encode it separately)
categorical_cols = df.select_dtypes(include=["object"]).columns.tolist()
if "attack_type" in categorical_cols:
    categorical_cols.remove("attack_type")

print(f"[INFO] Categorical columns to encode: {categorical_cols}")

# One-hot encode categorical cols (safe for models)
df_enc = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

# Final feature matrix X and labels y_multi (multi-class) and y_bin (binary)
X = df_enc.drop(columns=["attack_type", "is_attack"])
y_multi = df_enc["attack_type"]
y_bin = df_enc["is_attack"]

# Encode multi-class labels to integers
le = LabelEncoder()
y_multi_enc = le.fit_transform(y_multi)
print(f"[INFO] Multi-class labels found: {list(le.classes_)}")